In [6]:
import matplotlib.pyplot as plt
import cv2 as cv
import numpy as np
import pylab
import imageio
from tqdm import tqdm
from joblib import Parallel, delayed
from videoio import videosave, videoread

In [2]:

filename = "9210f99e-da25-41ed-9930-433a24105445_preview.mp4"

In [2]:


first_frame = 0
end_frame = vid_0.count_frames()

vid_0 = imageio.get_reader(filename,  'ffmpeg')
ref_img = vid_0.get_data(first_frame)

if not os.path.exists('results_of_MR'):
    os.mkdir('results_of_MR')
    
first_frame, end_frame

(0, 5550)

In [3]:
def dispOpticalFlow( Image,Flow,Divisor,name):
    "Display image with a visualisation of a flow over the top. A divisor controls the density of the quiver plot."
    PictureShape = np.shape(Image)
    #determine number of quiver points there will be
    Imax = int(PictureShape[0]/Divisor)
    Jmax = int(PictureShape[1]/Divisor)
    #create a blank mask, on which lines will be drawn.
    mask = np.zeros_like(Image)
    for i in range(1, Imax):
        for j in range(1, Jmax):
            X1 = (i)*Divisor
            Y1 = (j)*Divisor
            X2 = int(X1 + Flow[X1,Y1,1])
            Y2 = int(Y1 + Flow[X1,Y1,0])
            X2 = np.clip(X2, 0, PictureShape[0])
            Y2 = np.clip(Y2, 0, PictureShape[1])
            #add all the lines to the mask
            mask = cv.line(mask, (Y1,X1),(Y2,X2), [255, 255, 255], 1)
    #superpose lines onto image
    
    Image1 = np.zeros((np.shape(Image)[0],np.shape(Image)[1],3),np.uint8)
    Image1[:,:,0] = Image
    Image1[:,:,1] = Image
    Image1[:,:,2] = Image
    
    Image1[:,:,1] = cv.add(Image1[:,:,1],mask)

    return Image1/255.0

In [4]:
def performOpticalFlowAndSave(frame):
    
    if not os.path.exists('results_of_MR/frame_'+str(frame)+'.png'):
    
        vid0 = imageio.get_reader(filename,  'ffmpeg') # hack around a bug in the implementation
        vid1 = imageio.get_reader(filename,  'ffmpeg')

        prev_frame_gray = cv.cvtColor(vid0.get_data(frame-1), cv.COLOR_BGR2GRAY)
        new_frame_gray = cv.cvtColor(vid1.get_data(frame), cv.COLOR_BGR2GRAY)

        # Calculates dense optical flow by Farneback method
        flow = cv.calcOpticalFlowFarneback(prev_frame_gray, new_frame_gray, 
                                           None,
                                           0.5, 3, 15, 3, 5, 1.2, 0)

        np.save('results_of_MR/np_flow_'+str(frame)+'.npy',flow)
        cv2.imwrite('results_of_MR/optical_flow_'+str(frame)+'.png',dispOpticalFlow(prev_frame_gray,flow,5,'test')*255)
        cv2.imwrite('results_of_MR/frame_'+str(frame)+'.png',prev_frame_gray)

    

In [5]:
#Parallel(n_jobs=-1)(delayed(performOpticalFlowAndSave)(i) for i in range(first_frame+1,end_frame+1))

In [6]:
filename

'9210f99e-da25-41ed-9930-433a24105445_preview.mp4'

In [7]:
'''
#if not os.path.exists('results_of_MR/frame_'+str(frame)+'.png'):
frame = int(4535)

vid0 = imageio.get_reader(filename,  'ffmpeg')
vid1 = imageio.get_reader(filename,  'ffmpeg')

prev_frame_gray = cv.cvtColor(vid0.get_data(frame-1), cv.COLOR_BGR2GRAY)
new_frame_gray = cv.cvtColor(vid1.get_data(frame), cv.COLOR_BGR2GRAY)

# Calculates dense optical flow by Farneback method
flow = cv.calcOpticalFlowFarneback(prev_frame_gray, new_frame_gray, 
                                   None,
                                   0.5, 3, 15, 3, 5, 1.2, 0)

np.save('results_of_MR/np_flow_'+str(frame)+'.npy',flow)
cv2.imwrite('results_of_MR/optical_flow_'+str(frame)+'.png',dispOpticalFlow(prev_frame_gray,flow,5,'test')*255)
cv2.imwrite('results_of_MR/frame_'+str(frame)+'.png',prev_frame_gray)
'''

"\n#if not os.path.exists('results_of_MR/frame_'+str(frame)+'.png'):\nframe = int(4535)\n\nvid0 = imageio.get_reader(filename,  'ffmpeg')\nvid1 = imageio.get_reader(filename,  'ffmpeg')\n\nprev_frame_gray = cv.cvtColor(vid0.get_data(frame-1), cv.COLOR_BGR2GRAY)\nnew_frame_gray = cv.cvtColor(vid1.get_data(frame), cv.COLOR_BGR2GRAY)\n\n# Calculates dense optical flow by Farneback method\nflow = cv.calcOpticalFlowFarneback(prev_frame_gray, new_frame_gray, \n                                   None,\n                                   0.5, 3, 15, 3, 5, 1.2, 0)\n\nnp.save('results_of_MR/np_flow_'+str(frame)+'.npy',flow)\ncv2.imwrite('results_of_MR/optical_flow_'+str(frame)+'.png',dispOpticalFlow(prev_frame_gray,flow,5,'test')*255)\ncv2.imwrite('results_of_MR/frame_'+str(frame)+'.png',prev_frame_gray)\n"

In [8]:
#plt.imshow(dispOpticalFlow(prev_frame_gray,flow,5,'test')*255)

In [9]:
plt.imshow(prev_frame_gray)

NameError: name 'prev_frame_gray' is not defined

In [10]:
plt.imshow(new_frame_gray)

NameError: name 'new_frame_gray' is not defined

In [11]:
INTERVAL = 500
num_chunks = int(np.ceil((end_frame-first_frame)/INTERVAL))
print(num_chunks)

12


In [12]:
for chunk in range(num_chunks):
    
    first_frame_chunk = first_frame+chunk*INTERVAL
    end_frame_chunk = first_frame+(chunk+1)*INTERVAL
    if end_frame_chunk > end_frame:
        end_frame_chunk = end_frame

    opt_flow = np.zeros((end_frame_chunk-first_frame_chunk,ref_img.shape[0],ref_img.shape[1],ref_img.shape[2]),np.uint8)
    video_np = np.zeros((end_frame_chunk-first_frame_chunk,ref_img.shape[0],ref_img.shape[1],ref_img.shape[2]),np.uint8)

    kk10 = 0 
    for frame in tqdm(range(first_frame_chunk+1,end_frame_chunk+1)):
        img_cv = cv2.imread('results_of_MR/frame_'+str(frame)+'.png')
        opt_cv = cv2.imread('results_of_MR/optical_flow_'+str(frame)+'.png')

        opt_flow[kk10,:,:,:] = opt_cv
        video_np[kk10,:,:,:] = img_cv
        kk10+=1
    
    videosave("out_"+str(chunk)+".mp4", opt_flow/255)
    videosave("vid_"+str(chunk)+".mp4", video_np/255)

100%|██████████| 50/50 [00:00<00:00, 67.77it/s]


In [ ]:
'''
# Computes the magnitude and angle of the 2D vectors
magnitude, angle = cv.cartToPolar(flow[..., 0], flow[..., 1])

# Sets image hue according to the optical flow 
# direction
mask[..., 0] = angle * 180 / np.pi / 2

# Sets image value according to the optical flow
# magnitude (normalized)
mask[..., 2] = cv.normalize(magnitude, None, 0, 255, cv.NORM_MINMAX)

# Converts HSV to RGB (BGR) color representation
rgb = cv.cvtColor(mask, cv.COLOR_HSV2BGR)

#opt_flow[kk10,:,:,:] = dispOpticalFlow(prev_frame_gray,flow,5,'test')
#video_np[kk10,:,:,0] = prev_frame_gray
#video_np[kk10,:,:,1] = prev_frame_gray
#video_np[kk10,:,:,2] = prev_frame_gray
'''
